<a href="https://colab.research.google.com/github/wwfish/object_detection_lesson/blob/main/01_Traing_Tensorflow_August_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker-nightly
!pip3 install -q pycocotools

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
# Confirm TF Version
print(tf.__version__)

2.8.2


In [ ]:
# Download data
!gdown 1KLL5rL76q0SnmkoRgT1b_ncPox-_cEvL #combined
!unzip -q combined.zip

Downloading...
From: https://drive.google.com/uc?id=1KLL5rL76q0SnmkoRgT1b_ncPox-_cEvL
To: /content/combined.zip
100% 6.10G/6.10G [00:25<00:00, 242MB/s]


In [ ]:
# Load datasets
train_data = object_detector.DataLoader.from_pascal_voc(
    'train',
    'train',
   ['ball', 'coals', 'bicycle', 'beetle', 'beer', 'bee', 'bed', 'beard', 'bat', 'bag', 'axe', 'arm', 'ant', 'butterfly', 'bug', 'camel', 'bus', 'boy', 'boat', 'cake', 'box', 'bowl', 'bottle', 'book', 'bird', 'cloud', 'can', 'clothes', 'cat', 'car', 'clock', 'chicken', 'chair', 'caterpillar', 'canoe', 'fire', 'dish', 'cow', 'dog', 'face', 'eye', 'eagle', 'duck', 'door', 'crocodile', 'dolphin', 'furniture', 'house', 'horse', 'girl', 'fruit', 'head', 'hat', 'hand', 'hair', 'guitar', 'goat', 'frog', 'foot', 'food', 'flower', 'fish', 'phone', 'mirror', 'mountain', 'meat', 'motorbike', 'mug', 'nappy', 'man', 'pig', 'pen', 'owl', 'nose', 'mouth', 'lizard', 'knife', 'kangaroo', 'jeans', 'table', 'shoe', 'road', 'swimmers', 'pool', 'tap', 'snake', 'skirt', 'sink', 'shirt', 'shark', 'rabbit', 'pillow', 'wave', 'tree', 'woman', 'vegetable', 'truck', 'window', 'turkey', 'train', 'toy', 'towel', 'toilet', 'rock', 'creek', 'grass', 'shrub']
)

validation_data = object_detector.DataLoader.from_pascal_voc(
    'valid',
    'valid',
   ['ball', 'coals', 'bicycle', 'beetle', 'beer', 'bee', 'bed', 'beard', 'bat', 'bag', 'axe', 'arm', 'ant', 'butterfly', 'bug', 'camel', 'bus', 'boy', 'boat', 'cake', 'box', 'bowl', 'bottle', 'book', 'bird', 'cloud', 'can', 'clothes', 'cat', 'car', 'clock', 'chicken', 'chair', 'caterpillar', 'canoe', 'fire', 'dish', 'cow', 'dog', 'face', 'eye', 'eagle', 'duck', 'door', 'crocodile', 'dolphin', 'furniture', 'house', 'horse', 'girl', 'fruit', 'head', 'hat', 'hand', 'hair', 'guitar', 'goat', 'frog', 'foot', 'food', 'flower', 'fish', 'phone', 'mirror', 'mountain', 'meat', 'motorbike', 'mug', 'nappy', 'man', 'pig', 'pen', 'owl', 'nose', 'mouth', 'lizard', 'knife', 'kangaroo', 'jeans', 'table', 'shoe', 'road', 'swimmers', 'pool', 'tap', 'snake', 'skirt', 'sink', 'shirt', 'shark', 'rabbit', 'pillow', 'wave', 'tree', 'woman', 'vegetable', 'truck', 'window', 'turkey', 'train', 'toy', 'towel', 'toilet', 'rock', 'creek', 'grass', 'shrub']
)

test_data = object_detector.DataLoader.from_pascal_voc(
    'valid',
    'valid',
   ['ball', 'coals', 'bicycle', 'beetle', 'beer', 'bee', 'bed', 'beard', 'bat', 'bag', 'axe', 'arm', 'ant', 'butterfly', 'bug', 'camel', 'bus', 'boy', 'boat', 'cake', 'box', 'bowl', 'bottle', 'book', 'bird', 'cloud', 'can', 'clothes', 'cat', 'car', 'clock', 'chicken', 'chair', 'caterpillar', 'canoe', 'fire', 'dish', 'cow', 'dog', 'face', 'eye', 'eagle', 'duck', 'door', 'crocodile', 'dolphin', 'furniture', 'house', 'horse', 'girl', 'fruit', 'head', 'hat', 'hand', 'hair', 'guitar', 'goat', 'frog', 'foot', 'food', 'flower', 'fish', 'phone', 'mirror', 'mountain', 'meat', 'motorbike', 'mug', 'nappy', 'man', 'pig', 'pen', 'owl', 'nose', 'mouth', 'lizard', 'knife', 'kangaroo', 'jeans', 'table', 'shoe', 'road', 'swimmers', 'pool', 'tap', 'snake', 'skirt', 'sink', 'shirt', 'shark', 'rabbit', 'pillow', 'wave', 'tree', 'woman', 'vegetable', 'truck', 'window', 'turkey', 'train', 'toy', 'towel', 'toilet', 'rock', 'creek', 'grass', 'shrub']
)

In [ ]:
# Load model spec
spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite2', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite2/feature-vector/1', 
  model_dir='/content/checkpoints',
  hparams={'max_instances_per_image': 8000})

In [ ]:
# Train model
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, epochs=50, validation_data=validation_data)

Epoch 1/50
5681/5681 [==============================] - 2510s 433ms/step - det_loss: 1.4124 - cls_loss: 1.0739 - box_loss: 0.0068 - reg_l2_loss: 0.0990 - loss: 1.5114 - learning_rate: 0.0090 - gradient_norm: 3.5321 - val_det_loss: 1.1127 - val_cls_loss: 0.7716 - val_box_loss: 0.0068 - val_reg_l2_loss: 0.1033 - val_loss: 1.2160
Epoch 2/50
5681/5681 [==============================] - 2466s 434ms/step - det_loss: 1.1707 - cls_loss: 0.8583 - box_loss: 0.0062 - reg_l2_loss: 0.1074 - loss: 1.2781 - learning_rate: 0.0100 - gradient_norm: 3.3303 - val_det_loss: 1.1325 - val_cls_loss: 0.8176 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.1111 - val_loss: 1.2436
Epoch 3/50
5681/5681 [==============================] - 2464s 434ms/step - det_loss: 1.1103 - cls_loss: 0.8082 - box_loss: 0.0060 - reg_l2_loss: 0.1147 - loss: 1.2250 - learning_rate: 0.0099 - gradient_norm: 3.2456 - val_det_loss: 1.0008 - val_cls_loss: 0.6740 - val_box_loss: 0.0065 - val_reg_l2_loss: 0.1179 - val_loss: 1.1187
Epoch 4/50
56

In [ ]:
# Evaluate model
model.evaluate(test_data)

In [ ]:
# Convert model
model.export(export_dir='.', tflite_filename='combined-50e.tflite')

In [ ]:
# Download model
from google.colab import files
files.download('combined-50e.tflite')